In [1]:
import pandas as pd
import numpy as np
import reverse_geocoder as rg

In [2]:
data = pd.read_csv("data/ufo_sightings_scrubbed.csv",)

C:\Users\Kristina\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5,9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
data = pd.read_csv("data/ufo_sightings_scrubbed.csv", dtype={"duration (seconds)": "string", "latitude": "string"})

There were warning while importing 'ufo_sightings_scrubbed.csv': `DtypeWarning: Columns (5,9) have mixed types.`. So I watched the columns "duration (seconds)" and "latitude". Found out that there are values 2\`, 8\`, 0.5\` in column "duration (seconds)" and value 33q.200088. 

The mark \` after numbers seems to be mark of minutes. Removed them. Changed column dtype to numeric(float64).

Value 33q.200088 should be 33.200088. I found the row with this value and Mescalero Indian Reservation location latitude is about the same as 33.200088. Changed column dtype to numeric(float64).

Also working with columns the "longitude" was actually "longitude ".

In [4]:
print('duration (seconds)')
for value in data['duration (seconds)']:
    try:
        float(value)
    except:
        print(value)
        
print('latitude')
for value in data['latitude']:
    try:
        float(value)
    except:
        print(value)


duration (seconds)
2`
8`
0.5`
latitude
33q.200088


In [5]:
data = data.replace('33q.200088', '33.200088')
data["latitude"] = pd.to_numeric(data["latitude"])

In [6]:
data = data.replace('2`', '2')
data = data.replace('8`', '8')
data = data.replace('0.5`', '0.5')
data["duration (seconds)"] = pd.to_numeric(data["duration (seconds)"])

In [7]:
data = data.rename(columns={'longitude ': 'longitude'})

Now need to decide, what to do with `null` values.

Since there is no null values in "latitude" and "longitude" columns it is possible to find values for columns "state" and "country". Found that there is ["Reverse Geocoder" Python library](https://github.com/thampiman/reverse-geocoder) where I give coordinates and get data about the place.

I checked with Reverse Geocoder, if there are any values from "country" that does not correspond to given coordinates with code:
```
from collections import Counter

coordinates = []

for i in data[data['country'].notnull()].index:
    coordinates.append((data.at[i, 'latitude'], data.at[i, 'longitude']))

results = rg.search(coordinates)  

result_i = 0
count = 0
wrong = []
for i in data[data['country'].notnull()].index:
    if (results[result_i].get('cc').lower() != data.at[i, 'country'].lower()):
        print('Koordinaatide järgi:', (results[result_i].get('cc').lower()))
        print('Andmetes märgitud:',data.at[i, 'country'])
        print(data.iloc[i])
        print(i)
        wrong.extend([data.at[i, 'country'] + '_' + results[result_i].get('cc').lower()]) 
        count += 1
    result_i += 1
    
Counter(wrong)
```
The code looked about 70,000 rows and found only 93 not correspondending places. I chekced some not correspondending places manually and came out that actually the country in data is correct, but the places are so near to country borders that sometimes Reverse Geocoder thought it should be neighbour country. For example Curlew, Washington is in US, but when I gave coordinates to Reverse Geocoder, it thought it has to be Canada. Maybe because Curlew is so small place thath Reverse Geocoder does not it in its database. Result of not correspondending places, where on first place is abbreviation of country in our data and second is from Reverse Geocoder {'us_pr': 25, 'us_ca': 15, 'ca_us': 45, 'us_ru': 3, 'us_mx': 5}.  us - USA, pr - Puerto Rico, ca - Canada, ru - Russia, mx - Mexico.
It means that I can trust the marked countries in our data and I can use Reverse Geocoder for NaN values. There could be some mistakes, but the amount of them can not be big. Also I trust now values of "state" columns.

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80332 entries, 0 to 80331
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   datetime              80332 non-null  object 
 1   city                  80332 non-null  object 
 2   state                 74535 non-null  object 
 3   country               70662 non-null  object 
 4   shape                 78400 non-null  object 
 5   duration (seconds)    80332 non-null  float64
 6   duration (hours/min)  80332 non-null  object 
 7   comments              80317 non-null  object 
 8   date posted           80332 non-null  object 
 9   latitude              80332 non-null  float64
 10  longitude             80332 non-null  float64
dtypes: float64(3), object(8)
memory usage: 6.7+ MB


In [9]:
states = {
    'Alaska': 'ak',
    'Alabama': 'al',
    'Arkansas': 'ar',
    'Arizona': 'az',
    'California': 'ca',
    'Colorado': 'co',
    'Connecticut': 'ct',
    'District of Columbia': 'dc',
    'Delaware': 'de',
    'Florida': 'fl',
    'Georgia': 'ga',
    'Hawaii': 'hi',
    'Iowa': 'ia',
    'Idaho': 'id',
    'Illinois': 'il',
    'Indiana': 'in',
    'Kansas': 'ks',
    'Kentucky': 'ky',
    'Louisiana': 'la',
    'Massachusetts': 'ma',
    'Maryland': 'md',
    'Maine': 'me',
    'Michigan': 'mi',
    'Minnesota': 'mn',
    'Missouri': 'mo',
    'Mississippi': 'ms',
    'Montana': 'mt',
    'North Carolina': 'nc',
    'North Dakota': 'nd',
    'Nebraska': 'ne',
    'New Hampshire': 'nh',
    'New Jersey': 'nj',
    'New Mexico': 'nm',
    'Nevada': 'nv',
    'New York': 'ny',
    'Ohio': 'oh',
    'Oklahoma': 'ok',
    'Oregon': 'or',
    'Pennsylvania': 'pa',
    'Rhode Island': 'ri',
    'South Carolina': 'sc',
    'South Dakota': 'sd',
    'Tennessee': 'tn',
    'Texas': 'tx',
    'Utah': 'ut',
    'Virginia': 'va',
    'Vermont': 'vt',
    'Washington': 'wa',
    'Wisconsin': 'wi',
    'West Virginia': 'wv',
    'Wyoming': 'wy'
}

In [10]:
#Get all rows where country or state is null. Source: https://stackoverflow.com/questions/23330654/update-a-dataframe-in-pandas-while-iterating-row-by-row
#Fill state onyl when it is US?
coordinates = []
result_i = 0
for i in data[data['country'].isnull() |  ((data['state'].isna()) & (data['country'] == 'us'))].index:
    
    coordinates.append((data.at[i, 'latitude'], data.at[i, 'longitude']))
    
results = rg.search(coordinates)

for i in data[data['country'].isnull() |  ((data['state'].isna()) & (data['country'] == 'us'))].index:
    if pd.isna(data.at[i, 'country']):
        data.at[i, 'country'] = results[0].get('cc').lower()
        
    if (pd.isna(data.at[i, 'state'])) & (data.at[i, 'country'] == 'us'):
        data.at[i, 'country'] = states.get(results[0].get('admin1'))
        
    result_i += 1

Loading formatted geocoded file...


Now need to do something with shape values. Since the value "light" is most represented, 16,565 times and second most represented value is "triangle", 7865 times, then it seems to okay use "light" value for ~2000 NaN values. Also some shapes should be renamed like "round" to "circle" , because the meaning of them is same.

In [11]:
data['shape'] = data['shape'].replace(np.nan, 'light')
data['shape'] = data['shape'].replace('round', 'circle')
data['shape'] = data['shape'].replace('changed', 'changing')

Saving to new .csv file.

In [13]:
data.to_csv('ufo_sightings_cleaned.csv', encoding='utf-8', index=False)

In [14]:
data_new = pd.read_csv("data/ufo_sightings_scrubbed.csv")